<a href="https://colab.research.google.com/github/amrutask/Hate_Speech_Detection/blob/master/code/LSTM_random_embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))


Saving cleaned_tweets.csv to cleaned_tweets (1).csv
User uploaded file "cleaned_tweets.csv" with length 4298346 bytes


In [2]:
from keras.preprocessing.text import Tokenizer
import numpy as np
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.optimizers import RMSprop

from keras.layers import Input, Embedding, Activation, Flatten, Dense
from keras.layers import Conv1D, MaxPooling1D, Dropout
from keras.models import Model
from sklearn.model_selection import KFold
from keras.layers import Dense, LSTM, Dropout, Bidirectional, Embedding, GRU
from keras.callbacks import ModelCheckpoint, EarlyStopping


Using TensorFlow backend.


In [3]:
data=pd.read_csv("cleaned_tweets.csv")
tweets = data.iloc[:,7].values
labels= data.iloc[:,5].values
print(tweets[0:5])
labels[0:5]
tk = Tokenizer(num_words=None, char_level=False)
tk.fit_on_texts(tweets)

['rt as a woman you shouldn t complain about cleaning up your house amp as a man you should always take the trash out'
 'rt boy dats cold tyga dwn bad for cuffin dat hoe in the 1st place'
 'rt dawg rt you ever fuck a bitch and she start to cry you be confused as shit'
 'rt g anderson based she look like a tranny'
 'rt the shit you hear about me might be true or it might be faker than the bitch who told it to ya 57361']


In [34]:
#tk.word_index = unique_words.copy()
#print(tk.word_index)

tweet_data = tk.texts_to_sequences(tweets)

print("Size of the data:", len(tweet_data))

Size of the data: 24783


In [5]:
tweet_data = pad_sequences(tweet_data, maxlen=200, padding='post')
tweet_data[0]

array([   4,   80,    2,  396,    6, 1155,   10, 1113,   67, 3571,   34,
         33,  316,   65,   80,    2,   91,    6,  205,  169,  162,    5,
         55,   62,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,   

In [6]:
tweet_data = np.array(tweet_data, dtype='float32')
tweet_data.shape

(24783, 200)

In [0]:
from keras.utils import to_categorical

labels = to_categorical(labels)

In [22]:
vocab_size=len(tk.word_index)
print("Vocabulary size:", vocab_size)

embedding_size = 32

Vocabulary size: 22653


In [29]:
input_size = 200
vocab_size = len(tk.word_index)


optimizer = 'rmsprop'
loss = 'categorical_crossentropy'

from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(tweet_data, labels, test_size=0.2, random_state=0)
  
model = Sequential()
model.add(Embedding(vocab_size+1,
                    embedding_size,
                    input_length=input_size))

model.add(Bidirectional(LSTM(100)))
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])  
model.fit(X_train, Y_train,
       validation_data=(X_test, Y_test),
       batch_size=128,   
       epochs=20)

Train on 19826 samples, validate on 4957 samples
Epoch 1/20
19826/19826 [==============================] - 166s 8ms/step - loss: 0.5830 - acc: 0.7988 - val_loss: 0.4247 - val_acc: 0.8576
Epoch 2/20
19826/19826 [==============================] - 126s 6ms/step - loss: 0.3473 - acc: 0.8866 - val_loss: 0.3356 - val_acc: 0.8856
Epoch 3/20
19826/19826 [==============================] - 107s 5ms/step - loss: 0.3025 - acc: 0.9041 - val_loss: 0.3339 - val_acc: 0.8919
Epoch 4/20
19826/19826 [==============================] - 228s 12ms/step - loss: 0.3089 - acc: 0.9090 - val_loss: 0.3361 - val_acc: 0.8925
Epoch 5/20
19826/19826 [==============================] - 228s 12ms/step - loss: 0.2564 - acc: 0.9164 - val_loss: 0.3412 - val_acc: 0.8886
Epoch 6/20
19826/19826 [==============================] - 171s 9ms/step - loss: 0.2504 - acc: 0.9187 - val_loss: 0.3576 - val_acc: 0.8840
Epoch 7/20
19826/19826 [==============================] - 102s 5ms/step - loss: 0.2315 - acc: 0.9220 - val_loss: 0.3582 -

In [30]:
score = model.evaluate(X_test, Y_test, verbose=0) 
print("Test Accuracy: ", score[1])

Test Accuracy:  0.8240871495336733


In [0]:
pred= model.predict(X_test)

pred_arr = np.zeros(pred.shape[0])
actual_arr = np.zeros(Y_test.shape[0])

for i in range(pred.shape[0]):
  pred_arr[i]=np.argmax(pred[i])
  actual_arr[i]=np.argmax(Y_test[i])

In [32]:
from sklearn.metrics import confusion_matrix

confusion_matrix(actual_arr, pred_arr)

array([[ 130,  125,   24],
       [ 424, 3273,  155],
       [  42,  102,  682]])

In [33]:
from sklearn.metrics import classification_report
report = classification_report( actual_arr, pred_arr)
print(report)

              precision    recall  f1-score   support

         0.0       0.22      0.47      0.30       279
         1.0       0.94      0.85      0.89      3852
         2.0       0.79      0.83      0.81       826

   micro avg       0.82      0.82      0.82      4957
   macro avg       0.65      0.71      0.67      4957
weighted avg       0.87      0.82      0.84      4957

